In [1]:
import rpy2.robjects as ro
import pandas as pd
import json

r = ro.r
r['source']('ParseNFLPlayers.R')

parse_nfl_player_r = ro.globalenv['get_player_data']
parse_position_players_r = ro.globalenv['get_position_players']

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


R[write to console]: ── Attaching packages ──────────────────────────────────────── nflverse 1.0.1 ──

R[write to console]: ✔ nflfastR 4.3.0     ✔ nflreadr 1.1.3
✔ nflseedR 1.0.2     ✔ nflplotR 1.0.0
✔ nfl4th   1.0.1     

R[write to console]: ──────────────────────────────────────────────────────────────── Ready to go! ──

R[write to console]: Registered S3 method overwritten by 'tune':
  method                   from   
  required_pkgs.model_spec parsnip

R[write to console]: ── Attaching packages ────────────────────────────────────── tidymodels 0.1.4 ──

R[write to console]: ✔ broom        0.7.12     ✔ recipes      0.2.0 
✔ dials        0.1.0      ✔ rsample      0.1.1 
✔ dplyr        1.0.8      ✔ tibble       3.1.6 
✔ ggplot2      3.3.5      ✔ tidyr        1.2.0 
✔ infer        1.0.0      ✔ tune         0.1.6 
✔ modeldata    0.1.1      ✔ workflows    0.2.4 
✔ parsnip      0.1.7      ✔ workflowsets 0.1.0 
✔ purrr        0.3.4      ✔ yardstick    0.0.9 

R[write to console]: ── Confl

In [2]:
# Get the data from the R function
string_json_pmahomes = parse_nfl_player_r("Patrick Mahomes")[0]
# Parse the string into a JSON
json_pmahomes = json.loads(string_json_pmahomes)
print(json_pmahomes)

{'team': ['KC'], 'position': ['QB'], 'depth_chart_position': ['QB'], 'jersey_number': [15], 'status': ['Active'], 'full_name': ['Patrick Mahomes'], 'first_name': ['Patrick'], 'last_name': ['Mahomes'], 'birth_date': ['1995-09-17'], 'height': ['75'], 'weight': ['227'], 'college': ['Texas Tech'], 'high_school': ['Whitehouse (TX)'], 'gsis_id': ['00-0033873'], 'espn_id': [3139477], 'sportradar_id': ['11cad59d-90dd-449c-a839-dddaba4fe16c'], 'yahoo_id': [30123], 'rotowire_id': [11839], 'pff_id': [11765], 'pfr_id': ['MahoPa00'], 'fantasy_data_id': [18890], 'sleeper_id': ['4046'], 'years_exp': [5], 'headshot_url': ['https://static.www.nfl.com/image/private/f_auto,q_auto/league/tlg2oo5yrk6mins3nxxe'], 'week': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21], 'completions': [27, 24, 27, 24, 33, 32, 20, 29, 20, 35, 23, 15, 20, 31, 23, 26, 27, 30, 33, 26], 'attempts': [36, 31, 44, 30, 54, 47, 35, 48, 37, 50, 37, 29, 24, 47, 30, 35, 44, 39, 44, 39], 'passing_yards': [337, 343,

In [3]:
# Turn the JSON into a dataframe
df = pd.json_normalize(json_pmahomes)
# Display the dataframe
df.head()

,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,...,team_division,team_color,team_color2,team_color3,team_color4,team_logo_wikipedia,team_logo_espn,team_wordmark,team_conference_logo,team_league_logo
0,[KC],[QB],[QB],[15],[Active],[Patrick Mahomes],[Patrick],[Mahomes],[1995-09-17],[75],...,[AFC West],[#E31837],[#FFB612],[#000000],[#e31837],[https://upload.wikimedia.org/wikipedia/en/thu...,[https://a.espncdn.com/i/teamlogos/nfl/500/kc....,[https://github.com/nflverse/nflfastR-data/raw...,[https://github.com/nflverse/nflfastR-data/raw...,[https://raw.githubusercontent.com/nflverse/nf...


In [4]:
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# This is an R "tibble" of the data
# We load this object from the R script we used earlier
player_stats_r = ro.globalenv['player_stats']
team_data_r = ro.globalenv['team_data']
player_stats_kicker_r = ro.globalenv['player_stats_kicker']
roster_r = ro.globalenv['roster']
official_player_stats_r = ro.globalenv['official_player_stats']

# Convert the tibble to a pandas data frame
with localconverter(ro.default_converter + pandas2ri.converter):
    df_player_stats = ro.conversion.rpy2py(player_stats_r)
    df_team_data = ro.conversion.rpy2py(player_stats_r)
    df_player_stats_kicker = ro.conversion.rpy2py(player_stats_kicker_r)
    df_roster = ro.conversion.rpy2py(roster_r)
    df_official_player_stats = ro.conversion.rpy2py(official_player_stats_r)


In [5]:
team_passing_tds = {}
for index, recent_team in df_official_player_stats.iterrows():
    if(df_official_player_stats["recent_team"][index] not in team_passing_tds.keys()):
        team_passing_tds[df_official_player_stats["recent_team"][index]] = 0
    team_passing_tds[df_official_player_stats["recent_team"][index]] += df_official_player_stats["passing_tds"][index]
print(team_passing_tds)

team_passing_yards = {}
for index, recent_team in df_official_player_stats.iterrows():
    if(df_official_player_stats["recent_team"][index] not in team_passing_yards.keys()):
        team_passing_yards[df_official_player_stats["recent_team"][index]] = 0
    team_passing_yards[df_official_player_stats["recent_team"][index]] += df_official_player_stats["passing_yards"][index]
print(team_passing_yards)

{'TB': 43, 'ARI': 27, 'PIT': 23, 'GB': 39, 'WAS': 21, 'BAL': 24, 'SEA': 30, 'HOU': 21, 'ATL': 20, 'NYJ': 17, 'LV': 23, 'KC': 37, 'LA': 41, 'LAC': 38, 'NE': 24, 'BUF': 36, 'CHI': 16, 'SF': 26, 'CAR': 14, 'TEN': 23, 'NO': 29, 'NYG': 15, 'MIN': 34, 'CLE': 21, 'JAX': 12, 'IND': 27, 'DET': 23, 'PHI': 20, 'MIA': 21, 'DEN': 20, 'DAL': 40, 'CIN': 36}
{'TB': 5383.0, 'ARI': 4619.0, 'PIT': 4017.0, 'GB': 4526.0, 'WAS': 3746.0, 'BAL': 4601.0, 'SEA': 3815.0, 'HOU': 3630.0, 'ATL': 3987.0, 'NYJ': 3625.0, 'LV': 4808.0, 'KC': 4937.0, 'LA': 4893.0, 'LAC': 5014.0, 'NE': 4098.0, 'BUF': 4450.0, 'CHI': 3635.0, 'SF': 4437.0, 'CAR': 3573.0, 'TEN': 3745.0, 'NO': 3437.0, 'NYG': 3463.0, 'MIN': 4450.0, 'CLE': 3619.0, 'JAX': 3674.0, 'IND': 3588.0, 'DET': 3884.0, 'PHI': 3585.0, 'MIA': 3936.0, 'DEN': 3856.0, 'DAL': 4963.0, 'CIN': 4806.0}


In [6]:
df_official_player_stats["percent_rec_tds"] = 0
for index, receving_tds in df_official_player_stats.iterrows():
    team = str(df_official_player_stats["recent_team"][index])
    df_official_player_stats["percent_rec_tds"][index] = df_official_player_stats["receiving_tds"][index] / team_passing_tds[team]

df_official_player_stats["percent_rec_yards"] = 0
for index, receving_yards in df_official_player_stats.iterrows():
    team = str(df_official_player_stats["recent_team"][index])
    df_official_player_stats["percent_rec_yards"][index] = df_official_player_stats["receiving_yards"][index] / team_passing_yards[team]

df_official_player_stats.sort_values(by='receiving_yards', ascending=False)

/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_29344/558310768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["percent_rec_tds"][index] = df_official_player_stats["receiving_tds"][index] / team_passing_tds[team]
/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_29344/558310768.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["percent_rec_yards"][index] = df_official_player_stats["receiving_yards"][index] / team_passing_yards[team]


,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,percent_rec_tds,percent_rec_yards
272,00-0033908,C.Kupp,17,LA,0,1,0.0,0,0.0,0.0,...,1,1.186472,0.323262,0.331846,0.717186,0.0,294.50,439.50,0.390244,0.397915
542,00-0036322,J.Jefferson,17,MIN,2,4,35.0,0,0.0,0.0,...,0,0.780299,0.294111,0.458190,0.761899,0.0,222.40,330.40,0.294118,0.363146
98,00-0031381,D.Adams,16,GB,0,0,0.0,0,0.0,0.0,...,0,0.969413,0.316801,0.370177,0.734326,0.0,221.30,344.30,0.282051,0.343129
625,00-0036900,J.Chase,17,CIN,0,0,0.0,0,0.0,0.0,...,0,0.899814,0.232330,0.369006,0.606799,0.0,223.60,304.60,0.361111,0.302747
472,00-0035719,D.Samuel,16,SF,1,2,24.0,1,0.0,0.0,...,0,1.365403,0.262053,0.289081,0.595437,0.0,261.96,338.96,0.230769,0.316655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,00-0036650,L.Rountree,8,LAC,0,0,0.0,0,0.0,0.0,...,0,1.000000,0.026786,-0.001931,0.038827,0.0,14.60,15.60,0.000000,-0.000199
123,00-0031800,T.Heinicke,16,WAS,321,494,3419.0,20,15.0,38.0,...,0,0.000000,0.041667,-0.024242,0.045530,0.0,221.86,222.86,0.000000,-0.000534
43,00-0029164,M.Remmers,1,KC,0,0,0.0,0,0.0,0.0,...,0,0.000000,0.029412,0.000000,0.044118,0.0,-0.20,0.80,0.000000,-0.000405
93,00-0031319,D.Spencer,4,DEN,0,0,0.0,0,0.0,0.0,...,0,-0.333333,0.034316,0.011005,0.059178,0.0,-0.30,0.70,0.000000,-0.000778


In [26]:
df_official_player_stats["rec_dom"] = 0
for index, receving_yards in df_official_player_stats.iterrows():
    df_official_player_stats["rec_dom"][index] = (1/3)*(df_official_player_stats["percent_rec_yards"][index] + df_official_player_stats["percent_rec_tds"][index] + df_official_player_stats["target_share"][index])

df_official_player_stats["rec_share"] = 0
df_official_player_stats["rec_share_%"] = 0
for index, receving_yards in df_official_player_stats.iterrows():
    df_official_player_stats["rec_share"][index] = (1/4)*(df_official_player_stats["percent_rec_yards"][index]) + (1/2)*(df_official_player_stats["percent_rec_tds"][index]) + (1/4)*(df_official_player_stats["target_share"][index])
    df_official_player_stats["rec_share_%"][index] = "{:.0%}".format(df_official_player_stats["rec_share"][index])


df_official_player_stats.sort_values(by='rec_share', ascending=False)

/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_29344/793526572.py:3: RuntimeWarning: invalid value encountered in double_scalars
  df_official_player_stats["rec_dom"][index] = (1/3)*(df_official_player_stats["percent_rec_yards"][index] + df_official_player_stats["percent_rec_tds"][index] + df_official_player_stats["target_share"][index])
/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_29344/793526572.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_official_player_stats["rec_dom"][index] = (1/3)*(df_official_player_stats["percent_rec_yards"][index] + df_official_player_stats["percent_rec_tds"][index] + df_official_player_stats["target_share"][index])
/var/folders/bv/rzcr2l7j0fg9_28jldmf2kv40000gn/T/ipykernel_29344/793526572.py:8: RuntimeWarning: invalid value encountered in

,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,percent_rec_tds,percent_rec_yards,rec_dom,rec_dom_2,rec_share,rec_share_%
272,00-0033908,C.Kupp,17,LA,0,1,0.0,0,0.0,0.0,...,0.717186,0.0,294.50,439.50,0.390244,0.397915,0.370474,0.375416,0.375416,38%
457,00-0035640,D.Metcalf,17,SEA,0,0,0.0,0,0.0,0.0,...,0.686479,0.0,169.30,244.30,0.400000,0.253473,0.310289,0.332717,0.332717,33%
348,00-0034753,M.Andrews,17,BAL,0,0,0.0,0,0.0,0.0,...,0.613314,0.0,194.10,301.10,0.375000,0.295805,0.309530,0.325897,0.325897,33%
399,00-0035216,D.Johnson,16,PIT,0,0,0.0,0,0.0,0.0,...,0.653602,0.0,167.40,274.40,0.347826,0.289022,0.305267,0.315907,0.315907,32%
625,00-0036900,J.Chase,17,CIN,0,0,0.0,0,0.0,0.0,...,0.606799,0.0,223.60,304.60,0.361111,0.302747,0.298729,0.314325,0.314325,31%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,00-0036971,T.Lawrence,17,JAX,359,602,3641.0,12,17.0,32.0,...,NaN,0.0,199.04,199.04,0.000000,0.000000,NaN,NaN,NaN,nan%
642,00-0036972,M.Jones,17,NE,352,521,3801.0,22,13.0,28.0,...,NaN,0.0,224.94,224.94,0.000000,0.000000,NaN,NaN,NaN,nan%
644,00-0036982,C.Golston,1,DAL,0,0,0.0,0,0.0,0.0,...,NaN,1.0,6.00,6.00,0.000000,0.000000,NaN,NaN,NaN,nan%
646,00-0036986,N.Wright,1,DAL,0,0,0.0,0,0.0,0.0,...,NaN,1.0,6.00,6.00,0.000000,0.000000,NaN,NaN,NaN,nan%


In [27]:
df_sorted_rec_share = df_official_player_stats.sort_values(by='rec_share', ascending=False)

df_sorted_rec_share.iloc[0:15, [1,51]]

,player_name,rec_share_%
272,C.Kupp,38%
457,D.Metcalf,33%
348,M.Andrews,33%
399,D.Johnson,32%
625,J.Chase,31%
542,J.Jefferson,31%
98,D.Adams,31%
381,H.Renfrow,30%
361,Dj.Moore,29%
87,B.Cooks,28%
